In [4]:
import pandas as pd

# Pensamentos sobre como detectar possíveis problemas a partir do número de palavras dos artigos publicados 

Esse notebook trabalha com dados extraídos a partir da renderização dos artigos nos dois sites do scielo (`new` e `classíco`), estes dados foram extraídos com a biblioteca `BeautifulSoup` e durante o processo alguns elementos podem não ter sido capturados ou excluídos intencionalmente [1].

O valor da similaridade entre os artigos analisados é extraído a partir da equiação $\eqref{eq:jaccard_eq}$, onde a quantidade de palavras resultantes da **interseção** (entre um conjunto não repetitivo de palavras dos dois sites) é dividido pela **união** entre todas as palavras não repetidas dos dois sites.

\begin{equation*}
J(A,B) = {{|A \cap B|}\over{|A \cup B|}} = {{|A \cap B|}\over{|A| + |B| - |A \cap B|}}
\label{eq:jaccard_eq} \tag{1}
\end{equation*}

Essa abordagem pode ser considerada ingênua porque não utiliza do contexto do texto que é analisado. Porém nós sabemos que o conteúdo avaliado entre os **dois** sites **tende** a ser o mesmo e que a falta de palavras pode revelar perda de dado, então essa escolha parece ser aquedada e suficientemente boa para uma aplicação direta e rápida.

Um ponto importante para considerar é que na etapa de extração de conteúdo $1$, os _scripts_ podem deixar de capturar alguns pequenos textos como por exemplo o endereço de _DOI_ no site novo ou algum texto de _copyright_ no site clássico. Sendo assim, quando um artigo possui poucas palavras, qualquuer perda de conteúdo (seja real ou por problema na etapa $1$) faz com que a similaridade caia drasticamente.


A par dessas situações, nós podemos concluir que:
1. Essa abordagem é boa para detectar problemas severos (_ex: falta um parágrafo_) em textos similares;
2. É ingenua para detectar problemas em textos com poucas palavras;
3. Textos com poucas palavras e baixa similaridade geralmente são _falsos positivos_;
4. Textos com muitas palavras e com baixa similaridade geralmente são _verdadeiros positivos_;
5. É necessário estabelecer uma linha de corte entre o número de palavras e os falsos positivos;

<!-- 
Dito isso, a análise por meio da equação $\eqref{eq:jaccard_eq}$ funciona muito bem para textos com uma quantidade de palavras relevante (talvez algo como $ X > 100$). Os textos que possuem poucas palavras podem sofrer com o problema da não extração correta de conteúdo na primeira etapa do processo [1].


Partindo então da premissa de que a similaridade por meio da equação de _jaccard_ funciona bem para caso com uma quantidade média de palavras
 -->

## Critérios utilizados

Nós consideraremos possíveis problemas no texto de um artigo quando a equação $\eqref{eq:unionminusinter}$ for verdadeira, ou seja quando a diferença entre a união e a interseção entre as palavras for maior do que `30` nós podemos considerar que há algo para avaliar nestes artigos. 

\begin{equation*}
|A \cup B| - |A \cap B| > 30
\label{eq:unionminusinter} \tag{1}
\end{equation*}


In [232]:
# Cria estilo para podermos clicar no texto
def make_clickable(val):
    return '<a target="_blank" href="https://www.scielo.br/scielo.php?script=sci_arttext&pid={}">{}</a>'.format(val, val)

In [233]:
# Lemos o dataset produzir por um script python
data = pd.read_csv(
    "output.csv",
    sep=";",
    names=["pid", "similarity", "qty_words_classic", "qty_words_new", "intersecion", "union", "distance"],
    na_filter=False,
)

In [245]:
# Criamos uma nova coluna no dataset a partir da diferença entre a união de palavras e a interseção entre elas
data["difference_union_intersection"] = data.apply(
    lambda row: abs(row.union - row.intersecion), axis=1
)

In [249]:
# Mostramos a tabela com artigos que possuem ((U(C N) - I(C N)) > 30)
data.query(
    "similarity < 0.9 and union_intersection > 30"
).sort_values(["similarity"], ascending=False)[
    ["pid", "similarity", "difference_union_intersection"]
].reset_index(drop=True).style.format({'pid': make_clickable})

,pid,similarity,difference_union_intersection
0,S0103-84782014001001721,0.763900,157
1,S1808-86942013000500648,0.720200,54
2,S1677-55382014000200279,0.713500,100
3,S0037-86822013000400529,0.705600,106
4,S1677-55382013000200293,0.698600,66
5,S0104-92242016000200125,0.694300,81
6,S2236-89062018000100040,0.689500,575
7,S1677-55382013000200295,0.673000,86
8,S0101-31732020000100039,0.672900,944
9,S1677-55382014000300435,0.671900,84


### Verificando a contra prova

Se verificarmos a quantidade de artigos com similaridade ($ S >= 70\%$) e com ($ DUI < 30 $), nós veremos que os artigos são a grande maiora dos casos, temos portanto uma alta similaridade com uma baixa diferença entre a união e a interseção ($ DUI $).

In [254]:
data.query(
    "similarity >= 0.7 and union_intersection < 30"
).shape

(1300, 9)

Podemos então verificar quais PIDs tiveram uma similaridade $ S < 70\%$. Observamos que não existe perda de conteúdo mas que na verdade existe a falha de captura das palavras na etapa $1$.

In [253]:
data.query(
    "similarity < 0.7 and union_intersection < 30"
).sort_values(["similarity"], ascending=False)[
    ["pid", "similarity", "difference_union_intersection"]
].reset_index(drop=True).style.format({'pid': make_clickable})

,pid,similarity,difference_union_intersection
0,S2236-46332013000100212,0.698100,16
1,S2236-46332012000200154,0.696400,17
2,S2175-78601980000200005,0.695700,7
3,S2175-78601980000100019,0.695700,7
4,S1415-47141998000200177,0.695700,7
5,S1415-47142001000300164,0.692300,8
6,S2236-46332013000200163,0.690900,17
7,S2175-78601981000100071,0.690100,22
8,S1415-47142000000100183,0.689700,9
9,S1519-70772015000100119,0.687500,5
